Notebook equivalent of `fit_ensemble.py`
---

* This program trains the ensemble of CNN models reported in https://link.springer.com/article/10.1007/s42113-020-00073-z (https://osf.io/efjmq)
    * It trains a model/ensemble on a 180 images training set of a 360 images dataset
    * Makes predictions on
        1. the 90 images validation set (part of the same 360 images set)
        2. the 90 images test set (part of the same 360 images set)
        3. the 120 images set (a different set)
* Data required
    * File `mds_360.txt` with labels (in `../sanders_2018`)
    * Directory `360 Rocks/` with `*.jpg` images (in `../sanders_2018`)
    * File `mds_120.txt` with labels (in `../sanders_2018`)
    * Directory `120 Rocks/` with `*.jpg` images  (in `../sanders_2018`)
* other available data
    * Directory `120 Rock Images/` with 120 `*.png` images
    * Directory `Similarity Judgements Data/` with similarity labels for the "120 Rocks" set as individual textfiles for each of the 85 participants: `rocks_similarity_120_*.txt`
    * Directory `Categorization Data/` with category labels (1 = Igneous, 2 = Metamorphic, 4 = Mixed) for the "120 Rocks" set as individual textfiles for each of the 85 participants: `rocks_similarity_120_*_*.txt`
    * File `MDS/mds_120_supplemental_dims.txt`
    
    
#### **Update 2022/05/31: Additional necessary data added to `../sanders_2018` from here: https://osf.io/d6b9y/**

   * Rocks dataset was created in 2017 here: https://link.springer.com/article/10.3758/s13428-017-0884-8 (https://osf.io/w64fv)
   * Further work in Sanders' 2018 doctoral thesis https://scholarworks.iu.edu/dspace/handle/2022/22415 (https://osf.io/d6b9y)
        * includes the relevant additional data such as the 360 rocks images set
        * includes the same script `fit_ensemble.py` (identical version)

In [1]:
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import backend as K

import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

nPixels = 224

nTest = 90

## Categories

In [2]:
categories = [i for i in range(30) for j in range(12)] # creates 360 list items like so: [0, 0, 0, 0, ... 29, 29, 29, 29]

## Functions

In [3]:
def load_images(directory, nPixels, preprocesser):
    """
    Creates array-like data from a directory with image files for usage with Keras.
    """
    
    X = []
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg"):
                img = load_img(os.path.join(subdir, file), target_size=(nPixels, nPixels))
                x = img_to_array(img)
                X.append(x)
    X = np.stack(X)
    X = preprocesser(X)
    return X

## Prepare 360 Rocks data

In [4]:
# load image files
X = load_images("../sanders_2018/360 Rocks", nPixels, lambda x: resnet50.preprocess_input(np.expand_dims(x, axis=0)).squeeze())

# load labels
mds_360 = np.loadtxt("../sanders_2018/mds_360.txt") # missing

# split data: train vs test
(X_train_, X_test, 
 Y_train_, Y_test, 
 categories_train_, categories_test) = train_test_split(X, 
                                                        mds_360, 
                                                        categories,
                                                        test_size=nTest,
                                                        stratify=categories, 
                                                        random_state=0)

# split train set again: train vs validate
(X_train, X_validate, 
 Y_train, Y_validate) = train_test_split(X_train_, 
                                         Y_train_, 
                                         test_size=nTest,
                                         stratify=categories_train_, 
                                         random_state=0)

## Prepare 120 Rocks data

no train, test, validate splits ...will be later used only for testing

In [5]:
# load image files
X_120 = load_images("../sanders_2018/120 Rocks", nPixels, lambda x: resnet50.preprocess_input(np.expand_dims(x, axis=0)).squeeze())

# load labels
Y_120 = np.loadtxt("../sanders_2018/mds_120.txt") # missing

## Hyperparameters

In [6]:
datagen = ImageDataGenerator(featurewise_center=False,
                    samplewise_center=False,
                    featurewise_std_normalization=False,
                    samplewise_std_normalization=False,
                    zca_whitening=False,
                    rotation_range=20,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    channel_shift_range=0.,
                    fill_mode='nearest',
                    cval=0.,
                    horizontal_flip=True,
                    vertical_flip=True)

nEpochs = 10
dropout = 0.5
nEnsemble = 2
          
nDense = 256
nLayers = 2
loglr = -2.2200654426745987

lr = 10 ** loglr
nDim = 8
batch_size = 90

## Train models and save checkpoints

In [7]:
# load model

arch = resnet50.ResNet50(include_top=False, pooling='avg')

2022-06-07 19:52:48.700578: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-07 19:52:48.700774: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mink): /proc/driver/nvidia/version does not exist
2022-06-07 19:52:48.720526: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
for layer in arch.layers:
    layer.trainable = False    
    
x = arch.output
x = Dropout(dropout)(x)
for lyr in range(nLayers):
    x = Dense(nDense, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout)(x)
x = Dense(nDim)(x)

model = Model(inputs=arch.input, outputs=x)

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                           

In [73]:
for e in range(nEnsemble):
    #Build model
    arch = resnet50.ResNet50(include_top=False, pooling='avg')
    for layer in arch.layers:
        layer.trainable = False    
    
    x = arch.output
    x = Dropout(dropout)(x)
    for lyr in range(nLayers):
        x = Dense(nDense, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(dropout)(x)
    x = Dense(nDim)(x)
    
    model = Model(inputs=arch.input, outputs=x)
    
    #Initial training
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))
    
    checkpoint1 = ModelCheckpoint('intermediate_model.hdf5', save_best_only=True)

    hist1 = model.fit(datagen.flow(X_train, Y_train, batch_size), 
                                steps_per_epoch=len(X_train) / batch_size,
                                epochs=nEpochs,
                                validation_data=(X_validate, Y_validate),
                                callbacks=[checkpoint1],
                                verbose=False)
    
    #Fine tuning
    model = load_model("intermediate_model.hdf5")
    
    for layer in model.layers:
        layer.trainable = True
    
    model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='mean_squared_error')
    
    batch_size = 30 #reduce the batch size so that the gradients of all layers can fit in memory
    
    checkpoint2 = ModelCheckpoint('ensemble_{}.hdf5'.format(e), save_best_only=True)
    
    hist2 = model.fit(datagen.flow(X_train, Y_train, batch_size), 
                                steps_per_epoch=len(X_train) / batch_size,
                                epochs=nEpochs,
                                validation_data=(X_validate, Y_validate),
                                callbacks=[checkpoint2],
                                verbose=False)
    
    K.clear_session() #Clear tensorflow session to prevent memory issues

## Load checkpoints and get predictions for validation and test sets

seems to be not sufficient / does not load to complete model as the results from an ensemble loaded this way are very poor (see also MDS notebook)

In [ ]:
checkpoints_dir = ""

validate_pred = np.zeros((nEnsemble, nTest, nDim))
test_pred = np.zeros((nEnsemble, nTest, nDim))
rocks_120_pred = np.zeros((nEnsemble, 120, nDim))

for e in range(nEnsemble):
    model = load_model(checkpoints_dir + "ensemble_{}.hdf5".format(e))
    validate_pred[e,:] = model.predict(X_validate)
    test_pred[e,:] = model.predict(X_test)
    rocks_120_pred[e,:] = model.predict(X_120)
    
    K.clear_session()

validate_prediction = np.mean(validate_pred, 0)
test_prediction = np.mean(test_pred, 0)
rocks_120_prediction = np.mean(rocks_120_pred, 0)

3/3 [==============================] - 5s 1s/step


## Get MSE

In [ ]:
print(mean_squared_error(Y_validate, validate_prediction))
print(mean_squared_error(Y_test, test_prediction))
print(mean_squared_error(Y_120, rocks_120_prediction))

## Get R²

In [ ]:
print(r2_score(Y_validate, validate_prediction))
print(r2_score(Y_test, test_prediction))
print(r2_score(Y_120, rocks_120_prediction))

## Save predictions to file

In [ ]:
cnn_ownpred_file = "CNN Predictions/MDS Dimensions/cnn_own_predicted_mds_120.txt"

np.savetxt(cnn_ownpred_file, rocks_120_prediction)